In [77]:
import os
import json
import pandas as pd
from datetime import datetime, timedelta

directory = os.fsencode('data/wtt_matches')

In [128]:
players = []

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if not filename.endswith(".json"):
        continue

    path = os.path.join(f'data/wtt_matches', filename) 
    with open(path, 'r') as f:
        matches = json.load(f)

    for m in matches:
        for c in m['competitiors']:
            for p in c['players']:
                if not p['playerOrgCode']:
                    p['playerOrgCode'] = c['competitiorOrg']
                players.append(p)

In [129]:
pf = pd.DataFrame(players)
pf.drop(columns=['playerPosition'], inplace=True)
pf

,playerId,playerName,playerGivenName,playerFamilyName,playerOrgCode
0,115910,LIN Gaoyuan,None,None,CHN
1,137237,LIN Shidong,None,None,CHN
2,133848,YUAN Licen,None,None,CHN
3,135888,XIANG Peng,None,None,CHN
4,121404,FAN Zhendong,None,None,CHN
...,...,...,...,...,...
41069,111050,Alvaro ROBLES,Alvaro,ROBLES,ESP
41070,112221,Lily ZHANG,Lily,ZHANG,USA
41071,102445,Offiong EDEM,Offiong,EDEM,NGR
41072,111833,Suthasini SAWETTABUT,Suthasini,SAWETTABUT,THA


In [130]:
pf[pf.playerName == 'Lubomir PISTEJ']

,playerId,playerName,playerGivenName,playerFamilyName,playerOrgCode
9135,107445,Lubomir PISTEJ,Lubomir,PISTEJ,SVK
9261,107445,Lubomir PISTEJ,Lubomir,PISTEJ,SVK
36858,107445,Lubomir PISTEJ,Lubomir,PISTEJ,SVK
36961,107445,Lubomir PISTEJ,Lubomir,PISTEJ,SVK
37015,107445,Lubomir PISTEJ,Lubomir,PISTEJ,SVK
37057,107445,Lubomir PISTEJ,Lubomir,PISTEJ,SVK


In [131]:
pf[pf.playerId.isna()]
# only 2, manually fix

,playerId,playerName,playerGivenName,playerFamilyName,playerOrgCode


In [132]:
name_conflicts = pf.groupby('playerId')['playerName'].apply(set)
conf = pd.DataFrame(name_conflicts)
conf['cnt'] = name_conflicts.apply(len)
conf[conf.cnt > 1]


,playerName,cnt
playerId,,
100032,"{ABDEL-AZIZ Farah, Farah ABDELAZIZ}",2
100439,"{Ahmed SALEH, SALEH Ahmed}",2
100486,"{Gaston ALTO, ALTO Gaston}",2
100621,"{Tiago APOLONIA, APOLONIA Tiago}",2
100696,"{ASSAR Omar, Omar ASSAR}",2
...,...,...
200391,"{MENEZES ARAUJO Abimael, MENEZES Abimael}",2
200756,"{SARDA Mahayla Aime, SARDA Mahayla}",2
205401,"{ZENG Zhiying, Zhiying ZENG}",2


In [133]:
id_conflicts = pf.groupby('playerName')['playerId'].apply(set)
idconf = pd.DataFrame(id_conflicts)
idconf['cnt'] = id_conflicts.apply(len)
idconf[idconf.cnt > 1]
# These actually might be different people... 

,playerId,cnt
playerName,,
Confedera??o Brasileira de Tenis de Mesa 2,"{100115802, 60896}",2
LEE Daeun,"{135391, 132702}",2
Yang WANG,"{112735, 109995}",2


In [134]:
org_conflicts = pf.groupby('playerId')['playerOrgCode'].apply(set)
oconf = pd.DataFrame(org_conflicts)
oconf['cnt'] = org_conflicts.apply(len)
oconf[oconf.cnt > 1]


,playerOrgCode,cnt
playerId,,
100868,"{SVK/FRA, SVK/CZE, CZE/SVK, SVK, SVK/USA, LUX/...",7
102380,"{ENG, ENG/GER, GBR}",3
103163,"{USA, USA/SUI}",2
103903,"{ROU/ESP, ROU}",2
104359,"{SVK/SRB, SRB}",2
...,...,...
137627,"{AUS/WAL, AUS/SUI, AUS, SVK/AUS, ESP/AUS, NOR/...",7
146048,"{PER/ECU, PER}",2
200322,"{KAZ, LUX/KAZ}",2


In [142]:
cleaned = []
for id, rows in pf.groupby('playerId'):
    clrow = {
        'id': id,
        'org': None,
        'name': None,
    }
    for row in rows.itertuples():
        if not clrow['name']:
            clrow['name'] = row.playerName
        if not clrow['org']:
            clrow['org'] = row.playerOrgCode
        elif row.playerOrgCode:
            # take shorter country code, don't want doubles codes
            if len(row.playerOrgCode) < len(clrow['org']):
                clrow['org'] = row.playerOrgCode

    cleaned.append(clrow)
cf = pd.DataFrame(cleaned)

In [143]:
pf[pf.playerId.isin(cf[cf.org.isna()].id)]

,playerId,playerName,playerGivenName,playerFamilyName,playerOrgCode


In [144]:
cf.to_csv('data/wtt_cleaned/players.tsv')

,id,org,name
0,100001,IND,ANTHONY Amalraj
1,100032,EGY,ABDEL-AZIZ Farah
2,100079,NGR,ABIODUN Bode
3,100089,RSA,ABRAHAMS Luke
4,100115200,AUS,Table Tennis Australia Ltd. 1
...,...,...,...
1790,46172,GUA,Federación Nacional de Tenis de Mesa de Guatem...
1791,46173,PER,Federación Deportiva Peruana de Tenis de Mesa 9
1792,46175,CRC,Federación Costarricense de Tenis de Mesa 11
1793,60896,BRA,Confedera??o Brasileira de Tenis de Mesa 2
